In [1]:
# Calculations
import numpy as np
import scipy as sci
from scipy.stats import binom

# Interactive
from ipywidgets import widgets
from IPython.display import display

# Graphing
import plotly.plotly as py
import plotly.graph_objs as go

# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = 'testing_grounds.css'
HTML(open(css_file, "r").read())

In [2]:
def plotly_trace(f, min_x, max_x, name = '', num_values = 500):
    
    # Generate the x and y coordinates
    x_values = np.linspace(min_x, max_x, num_values)
    margins  = (x_values > 0) & (x_values < 1)
    y_values = np.where(margins, f(x_values), np.zeros([x_values.size]))

    # Generate the plotly trace and return it
    trace = go.Scatter(
        visible = False,
        x = x_values, 
        y = y_values,
        name = name
    )
    return trace

In [42]:
def calc_posterior(n, N_H):
    def to_return(values):
        return (n + 1) * binom.pmf(N_H, n, values)
    return to_return

def calc_prior():
    def to_return(values):
        return np.ones([values.size])
    return to_return

def generate_bayes_comparison(min_x, max_x, num_trials):

    # Generate data for the prior
    data_prior = [plotly_trace(calc_prior(), min_x, max_x, 'PRIOR')]
    data_prior[0]['visible'] = True

    # Generate data for the posterior
    data_posterior = [plotly_trace(calc_posterior(num_trials, h), min_x, max_x, 'POSTERIOR') for h in np.arange(10, 191, 1)]
    data_posterior[num_trials // 2]['visible'] = True

    return data_prior + data_posterior

data = generate_bayes_comparison(-1, 2, 200)

In [44]:
# Slider used to toggle visibility for each posterior
slide_heads_toggles = []

# Go over each trace and define sliders
for i in range(len(data)):
    if i == 0: continue
    head_toggle = {
        'method' : 'restyle',
        'args'   : ['visible', [False] * len(data)],
        'label'  : str(i + 9)
    }
    
    head_toggle['args'][1][i] = True # Toggle i'th trace to visible 
    head_toggle['args'][1][0] = True # Prior is always visible
    
    slide_heads_toggles.append(head_toggle)

sliders = [{
    'active'       : 100,
    'currentvalue' : {'prefix': 'Number of heads: '},
    'pad'          : {'t': 50},
    'steps'        : slide_heads_toggles}]

layout = {
    'sliders' : sliders,
    'xaxis'   : {'title': 'Probability of a Head'},
    'yaxis'   : {'title': 'Probability Density', 'autorange' : False, 'range' : [0, 15]},
    'title'   : 'Comparison Betweeen Prior and Posterior Distributions (Number of Tosses = 200)'
}

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='Bayes Statistics')